In [ ]:
import heapq 

In [2]:
class Node:
    def __init__(self, position, parent=None, action=None, path_cost=0):
        self.position = position
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost


class Problem:
    def __init__(self, maze, start, goals, terrain_cost):
        self.maze = maze
        self.start = start
        self.goals = goals
        self.terrain_cost = terrain_cost
        self.actions = {
            (-1, 0): "Up",
            (1, 0): "Down",
            (0, -1): "Left",
            (0, 1): "Right"
        }

In [3]:
def find_exit(maze, start, goals, terrain_cost):
    problem = Problem(maze, start, goals, terrain_cost)

    def manhattan_to_nearest_goal(pos):
        return min(abs(pos[0] - g[0]) + abs(pos[1] - g[1]) for g in problem.goals)

    def get_neighbors(pos):
        neighbors = []
        for move, action_name in problem.actions.items():
            neighbor = (pos[0] + move[0], pos[1] + move[1])
            if maze[neighbor[0]][neighbor[1]] != "#":  # no es pared
                neighbors.append((neighbor, action_name))
        return neighbors

    start_node = Node(start, path_cost=0)
    frontier = [(manhattan_to_nearest_goal(start), start_node)]
    heapq.heapify(frontier)
    reached = {start: start_node}

    while frontier:
        _, node = heapq.heappop(frontier)
        if node.position in problem.goals:
            return reconstruct_path(node)

        for neighbor, action in get_neighbors(node.position):
            terrain = maze[neighbor[0]][neighbor[1]]
            move_cost = problem.terrain_cost.get(terrain, 1)  # costo por tipo de terreno
            new_cost = node.path_cost + move_cost

            if neighbor not in reached or new_cost < reached[neighbor].path_cost:
                reached[neighbor] = Node(neighbor, parent=node, action=action, path_cost=new_cost)
                priority = new_cost + manhattan_to_nearest_goal(neighbor)
                heapq.heappush(frontier, (priority, reached[neighbor]))

    return None  # no hay salida

In [4]:
def reconstruct_path(node):
    path = []
    actions = []
    while node:
        path.append(node.position)
        actions.append(node.action)
        node = node.parent
    path.reverse()
    actions.reverse()
    return path, actions


def print_maze_with_path(maze, path):
    maze_copy = [row[:] for row in maze]  # copia para no modificar el original
    for r, c in path[1:-1]:
        maze_copy[r][c] = "."
    for row in maze_copy:
        print("".join(row))

In [5]:
maze = [
    ["#", "#", "#", "#", "#", "#", "#", "#", "#", "#"],
    ["#", "S", " ", " ", "#", " ", " ", "E", "~", "#"],
    ["#", " ", "#", " ", "#", "~", "#", " ", " ", "#"],
    ["#", " ", "#", " ", " ", " ", "#", "#", " ", "#"],
    ["#", " ", " ", "~", "#", " ", " ", " ", "E", "#"],
    ["#", "#", "#", "#", "#", "#", "#", "#", "#", "#"]
]

start = (1, 1)
goals = [(1, 7), (4, 8)]  # múltiples salidas
terrain_cost = {
    " ": 1,  # camino normal
    "~": 3,  # agua: más costoso
    "S": 1,
    "E": 1
}

path, actions = find_exit(maze, start, goals, terrain_cost)

print("Path to exit:", path)
print("Actions:", actions)
print("\nLaberinto con el camino marcado:")
print_maze_with_path(maze, path)

Path to exit: [(1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (3, 5), (4, 5), (4, 6), (4, 7), (4, 8)]
Actions: [None, 'Right', 'Right', 'Down', 'Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Right']

Laberinto con el camino marcado:
##########
#S..#  E~#
# #.#~#  #
# #...## #
#  ~#...E#
##########
